In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from unidecode import unidecode
import re

In [ ]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.ecb.europa.eu/press/pr/activities/mopo/html/index.en.html"
    
    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    date_list = []
    title_list = []
    link_list = []
    article_list = []

    # Fetch dates, names and links as lists
    dates = soup.find_all("div", attrs={'class':'date'})
    titles = soup.find_all("div", attrs={'class':'category'})
    links = soup.find_all('dd')

    for date in dates:
        date_list.append(date.text)
    
    for title in titles:
        title_list.append(title.text)

    for link in links:
        link_list.append('https://www.ecb.europa.eu/' + link.find('div', class_='title').find('a').get('href'))        

In [ ]:
# Create a dataframe with the lists
df = pd.DataFrame(list(zip(date_list, title_list, link_list)), columns=['Date', 'Title', 'Link'])

# Filter to keep only those with Title MONETARY POLICY DECISION and Date from 2021 onward
df = df[df['Title'] == 'MONETARY POLICY DECISION']
df = df[df['Date'].apply(lambda x: datetime.strptime(x, '%d %B %Y').year >= 2021)]

df.head()

In [ ]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; ru) Opera 8.0', 'Accept-Language': 'en-US, en;q=0.5'})

    # The base webpage URL
    url_list = df['Link'].tolist()
    article_list = []

    for link in url_list:
        try:
            # HTTP Request
            webpage = requests.get(link, headers=HEADERS)

            # Soup Object containing all data
            soup = BeautifulSoup(webpage.content, "html.parser")
            a_tag = soup.find('div', id='main-wrapper')
            text = a_tag.find('div', class_='section').text.strip()

        except: text = np.nan

        try: text = unidecode(text)
        except:
            text = np.nan
        article_list.append(text)

In [ ]:
# Define a regular expression pattern to match the date at the beginning of each text
date_pattern = r'^\d+\s+\w+\s+\d+'

# Remove the date from each text using re.sub()
cleaned_articles = [re.sub(date_pattern, '', text).lstrip() for text in article_list]

In [ ]:
# Add the cleaned articles to the DataFrame
df['Article'] = cleaned_articles

In [ ]:
# Change date format 11 April 2024 to 2024-04-11
df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x, '%d %B %Y').strftime('%Y-%m-%d'))

In [ ]:
# Filter to keep only those between July 2022 and March 2024
df = df[df['Date'].apply(lambda x: '2022-07-01' <= x <= '2024-03-31')]

# Remove duplicates
df = df.drop_duplicates()
df = df.reset_index(drop=True)
print(df.shape)
df.head()